## 第1回 FR Frontier ：ファッション画像における洋服の「色」分類

* [コンテスト詳細 ビッグデータ活用ならオプトDSL DeepAnalytics](https://deepanalytics.jp/compe/36)

## preprocess
Opencvをインストール

    conda install opencv

In [2]:
import numpy as np
import pandas as pd
import cv2
from skimage.feature import greycomatrix, greycoprops

In [18]:
import os, sys
current_dir = os.getcwd()
HOME_DIR = current_dir
#DATA_GIVEN_DIR = HOME_DIR+"/data/given/"
DATA_GIVEN_DIR = "/input/"

In [19]:
width = zz
height = 667

### Load Data

In [21]:
data_train = []
for i in range(0,12399):
    src = DATA_GIVEN_DIR + 'train/' + 'train_%i.jpg'%i
    img = cv2.imread(src)zz

    histR = cv2.calcHist([img],[0],None,[256],[0,256])
    histG = cv2.calcHist([img],[1],None,[256],[0,256])
    histB = cv2.calcHist([img],[2],None,[256],[0,256])

    histR = np.array(histR[1:])
    histG = np.array(histG[1:])
    histB = np.array(histB[1:])

    hist = np.concatenate([histR, histG, histB], axis=1)
    data_train.append(hist)

In [17]:
data_train = np.array(data_train)
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
dataXR = mms.fit_transform(data_train[:,:,0])
dataXG = mms.fit_transform(data_train[:,:,1])
dataXB = mms.fit_transform(data_train[:,:,2])
dataX = np.stack([dataXR, dataXG, dataXB], axis=2)
dataX.shape

IndexError: too many indices for array

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
dataXR = stdsc.fit_transform(data_train[:,:,0])
dataXG = stdsc.fit_transform(data_train[:,:,1])
dataXB = stdsc.fit_transform(data_train[:,:,2])
dataX = np.stack([dataXR, dataXG, dataXB], axis=2)
dataX.shape

In [ ]:
data_test = []
for i in range(0, 9801):
    src = DATA_GIVEN_DIR + 'test/' + 'test_%i.jpg'%i
    img = cv2.imread(src)

    histR = cv2.calcHist([img],[0],None,[256],[0,256])
    histG = cv2.calcHist([img],[1],None,[256],[0,256])
    histB = cv2.calcHist([img],[2],None,[256],[0,256])
    
    histR = np.array(histR[1:])
    histG = np.array(histG[1:])
    histB = np.array(histB[1:])

    hist = np.concatenate([histR, histG, histB], axis=1)
    data_test.append(hist)

In [ ]:
data_test = np.array(data_test)
dataXR = mms.fit_transform(data_test[:,:,0])
dataXG = mms.fit_transform(data_test[:,:,1])
dataXB = mms.fit_transform(data_test[:,:,2])
testX = np.stack([dataXR, dataXG, dataXB], axis=2)
testX.shape

In [ ]:
dataXR = stdsc.fit_transform(data_test[:,:,0])
dataXG = stdsc.fit_transform(data_test[:,:,1])
dataXB = stdsc.fit_transform(data_test[:,:,2])
testX = np.stack([dataXR, dataXG, dataXB], axis=2)
testX.shape

### train/valid 作成

In [ ]:
train_master = pd.read_table(DATA_GIVEN_DIR+'train_master.tsv', decimal='\t')
dataY = np.array(train_master['category_id'])
dataY = np.array(pd.get_dummies(dataY).astype('float32'))
dataY.shape

In [ ]:
trainX, validX = dataX[:10399], dataX[10399:]
trainY, validY = dataY[:10399], dataY[10399:]
trainX.shape, validX.shape, trainY.shape, validY.shape

## training

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Conv1D, BatchNormalization, MaxPooling1D,GlobalAveragePooling1D, Activation
from keras.optimizers import Nadam
from keras.layers.advanced_activations import PReLU

In [ ]:
batch_size=64

In [ ]:
nf=256
p=0.5

In [ ]:
def get_model():
    return [
    BatchNormalization(axis=1, input_shape=(255,3)),
    Conv1D(nf,3, padding='same'),
    PReLU(),
    BatchNormalization(axis=1),
    MaxPooling1D(),
    Conv1D(nf,3, padding='same'),
    PReLU(),
    BatchNormalization(axis=1),
    MaxPooling1D(),
    Conv1D(nf,3, padding='same'),
    PReLU(),
    BatchNormalization(axis=1),
    MaxPooling1D(),
    Conv1D(nf,3, padding='same'),
    PReLU(),
    BatchNormalization(axis=1),
    MaxPooling1D(),
    Conv1D(24,3, padding='same'),
    PReLU(),
    BatchNormalization(axis=1),
    Dropout(p),
    GlobalAveragePooling1D(),
    Activation('softmax')
    ]

In [ ]:
model = Sequential(get_model())

In [ ]:
model.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor="loss", patience=25)

In [ ]:
model.fit(trainX, trainY, epochs=50, batch_size=batch_size, validation_data=(validX, validY), 
          callbacks=[early_stop], )

### Inception1D

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.pooling import AveragePooling1D, GlobalAveragePooling1D

In [ ]:
def conv1d_bn(x, nb_filter, nb_row, subsample_length=1):
    x = Conv1D(nb_filter, nb_row, activation='relu', padding='same')(x)
    return BatchNormalization(axis=1)(x)

In [ ]:
def incep_block(x):
    branch1 = conv1d_bn(x, 32, 1, subsample_length=2)
    branch5 = conv1d_bn(x, 24, 1)
    branch5 = conv1d_bn(branch5, 32, 5, subsample_length=2)

    branch3dbl = conv1d_bn(x, 32, 1)
    branch3dbl = conv1d_bn(branch3dbl, 48, 3)
    branch3dbl = conv1d_bn(branch3dbl, 48, 3, subsample_length=2)

    branch_pool = AveragePooling1D(pool_size=3, strides=2, padding='same')(x)
    branch_pool = conv1d_bn(x, 16, 1)
    return concatenate([branch1, branch5, branch3dbl, branch_pool], axis=2)

In [ ]:
inp = Input((255,3)) 
x = BatchNormalization(axis=1)(inp)
x = incep_block(x)
x = incep_block(x)
x = incep_block(x)
x = Dropout(0.75)(x)
x = Conv1D(24, 3, padding='same')(x)
x = GlobalAveragePooling1D()(x)
outp = Activation('softmax')(x)

In [ ]:
model = Model([inp], outp)

In [ ]:
model.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(trainX, trainY, epochs=30, batch_size=batch_size, validation_data=(validX, validY))b

###  Using TensorBoard

In [ ]:
def get_model_with_param(num_layers, num_filters):
    model = Sequential()
    model.add(BatchNormalization(axis=1, input_shape=(255,3)))
    for _ in range(num_layers):
        model.add(Conv1D(num_filters,3, padding='same'))
        model.add(PReLU())
        model.add(BatchNormalization(axis=1))
        model.add(MaxPooling1D())
    model.add(Conv1D(24,3, padding='same'))
    model.add(PReLU())
    model.add(BatchNormalization(axis=1))
    model.add(Dropout(p))
    model.add(GlobalAveragePooling1D())
    model.add(Activation('softmax'))
    model.compile(Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
for num_layers in [4,5]:
    for num_filters in [512]:
        log_string = 'logs/3/nl={},nf={}'.format(num_layers, num_filters)
        tbcb = keras.callbacks.TensorBoard(log_dir=log_string)
        model = get_model_with_param(num_layers, num_filters)
        model.fit(trainX, trainY, epochs=30, batch_size=batch_size, validation_data=(validX, validY), callbacks=[tbcb])

## Predict

In [ ]:
test_features = model.predict(testX, batch_size=batch_size)

In [ ]:
test_labels = np.argmax(test_features, axis=1)
test_labels[:5]
#test_features[:5]
#array([20, 14, 17, 20, 23])

In [ ]:
filenames = []
for i in range(0, 9801):
    filenames.append("test_%i.jpg"%i)

In [ ]:
submission = pd.DataFrame({"Image": filenames,
                             "Label": test_labels})

In [ ]:
submission_file_name = 'submission_conv2_256_4_std.csv'
#np.savetxt(submission_file_name, subm, delimiter=',')
submission.to_csv(submission_file_name, index=False, header=None)

In [ ]:
from IPython.display import FileLink
FileLink(submission_file_name)

## Pseude Labeling

In [ ]:
comb_feat = np.concatenate([trainX, testX])

In [ ]:
test_labels_oh = np.array(pd.get_dummies(test_labels).astype('float32'))
comb_label = np.concatenate([trainY, test_labels_oh])

In [ ]:
model.fit(comb_feat, comb_label, epochs=30, batch_size=batch_size, validation_data=(validX, validY))